# Eat Safe, Love

## Notebook Set Up

In [7]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [8]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [12]:
# Find the establishments with a hygiene score of 20
# Use count_documents to display the number of documents in the result
# Display the first document in the results using pprint
query = {'scores.Hygiene':20}

document_count = establishments.count_documents(query)
print(document_count)

pprint(establishments.find_one(query))


0
None


In [ ]:
# Convert the result to a Pandas DataFrame
# Display the number of rows in the DataFrame
# Display the first 10 rows of the DataFrame
df = pd.DataFrame(establishments.find(query))
print(f"There are {len(df)} rows in this DataFrame.")
df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
# Use count_documents to display the number of documents in the result
# Display the first document in the results using pprint
second_query = {"LocalAuthorityName":{'$regex':"London"},"RatingValue":{'$gte':4}}

rating_count = establishments.count_documents(second_query)
print(f"There are {rating_count} establishments with a `RatingValue` greater than or equal to 4.")

pprint(establishments.find_one(second_query))

In [ ]:
# Convert the result to a Pandas DataFrame
# Display the number of rows in the DataFrame
# Display the first 10 rows of the DataFrame
rating_df = pd.DataFrame(establishments.find(second_query))
print(f"There are {len(rating_df)} rows.")
rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Print the results

pprint(establishments.find_one({'BusinessName': 'Penang Flavours'}, 
                               {'geocode.latitude', 'geocode.longitude'}))

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

third_query ={"RatingValue": 5, 
        "geocode.latitude": {"$gte": (latitude - degree_search), "$lte": (latitude + degree_search)},
        "geocode.longitude": {"$gte": (longitude - degree_search), "$lte": (longitude + degree_search)}}
sort = [("scores.Hygiene", 1)]
limit = 5

results = list(establishments.find(third_query).sort(sort).limit(limit))
pprint(results)

In [ ]:
# Convert result to Pandas DataFrame
hygiene_df = pd.DataFrame(results)
hygiene_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# Print the number of documents in the result
# Print the first 10 results
matches = {'$match': {"scores.Hygiene":0}}

groups = {"$group":{"_id":"$LocalAuthorityName",
                    "count":{ "$sum":1}}}

sorts = {"$sort": {"count": -1}}

pipeline = [matches, groups, sorts]
pipeline_results = list(establishments.aggregate(pipeline))

print(len(pipeline_results))

pprint(pipeline_results[0:10]) 



In [ ]:
# Convert the result to a Pandas DataFrame
# Display the number of rows in the DataFrame
# Display the first 10 rows of the DataFrame
authority_hygine_df = pd.DataFrame(pipeline_results)

print(len(authority_hygine_df))

authority_hygine_df.head(10)